In [15]:
import logging

import pandas as pd
import numpy as np

import soepy

Pre processing

In [2]:
def pre_process_data(data):
    """This function perform auxiliary calculations given a data set
    and generates additional variable needed for the calculation of 
    moments"""
    
    # Determine the education level given years of experience
    data["Educ_Level"] = 0
    data.loc[data["Years_of_Education"] == 11, "Educ_Level"] = 1
    data.loc[data["Years_of_Education"] == 12, "Educ_Level"] = 2

    # Determine the observed wage given period choice
    data["Wage_Observed"] = 0
    data.loc[data["Choice"] == 0, "Wage_Observed"] = data.loc[data["Choice"] == 0, "Period_Wage_N"]
    data.loc[data["Choice"] == 1, "Wage_Observed"] = data.loc[data["Choice"] == 1, "Period_Wage_P"]
    data.loc[data["Choice"] == 2, "Wage_Observed"] = data.loc[data["Choice"] == 2, "Period_Wage_F"]
    
    return data

Moments

In [3]:
def get_moments(data):
    # Initialize moments dictionary
    moments = dict()

    # Store moments in groups as nested dictionary
    for group in [
        "Wage_Distribution",
        "Choice_Probability",
    ]:
        moments[group] = dict()
    
    
    # Compute unconditional moments of the wage distribution
    info = data.groupby(["Period"])["Wage_Observed"].describe().to_dict()

    # Save mean and standard deviation of wages for each period
    # to Wage Distribution section of the moments dictionary
    for period in range(40): ## TO DO: Remove hard coded number
        moments["Wage_Distribution"][period] = []
        try:
            for label in ["mean", "std"]:
                moments["Wage_Distribution"][period].append(info[label][period])
        except KeyError:
            for i in range(2):
                moments["Wage_Distribution"][period].append(
                    0.0
                )
    
    
    # Compute unconditional moments of the choice probabilities
    info = data.groupby(["Period"])["Choice"].value_counts(normalize=True).to_dict()

    for period in range(40): ## TO DO: Remove hard coded number
        moments["Choice_Probability"][period] = []
        for choice in range(3):
            try:
                stat = info[(period, choice)]
            except KeyError:
                stat = 0.00
            moments["Choice_Probability"][period].append(stat)
        
    return moments

Weighting matrix

In [10]:
def get_weighting_matrix(data, num_agents_smm, num_samples):
    """Calculates the weighting matrix based on the
    moments of the observed data"""

    moments_sample = []

    # Collect n samples of moments
    for k in range(num_samples):
        data_sample = data.sample(n=num_agents_smm)

        moments_sample_k = get_moments(data_sample)

        moments_sample.append(moments_sample_k)

        k = +1

    # Append samples to a list of size num_samples
    # containing number of moments values each
    stats = []

    for moments_sample_k in moments_sample:
        stats.append(moments_dict_to_list(moments_sample_k))

    # Calculate sample variances for each moment
    moments_var = np.array(stats).var(axis=0)

    # Handling of nan
    moments_var[np.isnan(moments_var)] = np.nanmax(moments_var)

    # Handling of zero variances
    is_zero = moments_var <= 1e-10
    moments_var[is_zero] = 0.1

    # Construct weighting matrix
    weighting_matrix = np.diag(moments_var ** (-1))

    return weighting_matrix


def moments_dict_to_list(moments_dict):
    """This function constructs a list of available moments based on the moment dictionary."""
    moments_list = []
    for group in [
        "Wage_Distribution",
        "Choice_Probability",
    ]:
        for period in sorted(moments_dict[group].keys()):
            moments_list.extend(moments_dict[group][period])
    return moments_list


In [11]:
def update_optim_paras(model_params_init_file_name, optim_paras):
    """Update the parameters with values of 
    the current optimizer iteration"""

    model_params_df = pd.read_pickle(model_params_init_file_name)

    model_params_df.loc["const_wage_eq", "gamma_0s1"] = optim_paras.item(0)
    model_params_df.loc["const_wage_eq", "gamma_0s2"] = optim_paras.item(1)
    model_params_df.loc["const_wage_eq", "gamma_0s3"] = optim_paras.item(2)
    model_params_df.loc["exp_returns", "gamma_1s1"] = optim_paras.item(3)
    model_params_df.loc["exp_returns", "gamma_1s2"] = optim_paras.item(4)
    model_params_df.loc["exp_returns", "gamma_1s3"] = optim_paras.item(5)
    model_params_df.loc["exp_accm", "g_s1"] = optim_paras.item(6)
    model_params_df.loc["exp_accm", "g_s2"] = optim_paras.item(7)
    model_params_df.loc["exp_accm", "g_s3"] = optim_paras.item(8)
    model_params_df.loc["exp_deprec", "delta_s1"] = optim_paras.item(9)
    model_params_df.loc["exp_deprec", "delta_s2"] = optim_paras.item(10)
    model_params_df.loc["exp_deprec", "delta_s3"] = optim_paras.item(11)
    model_params_df.loc["disutil_work", "const_p"] = optim_paras.item(12)
    model_params_df.loc["disutil_work", "const_f"] = optim_paras.item(13)
    # TO DO: Enable flexible number of types
    model_params_df.loc["hetrg_unobs", "theta_p1"] = optim_paras.item(14)
    model_params_df.loc["hetrg_unobs", "theta_f1"] = optim_paras.item(15)
    model_params_df.loc["hetrg_unobs", "theta_p2"] = optim_paras.item(16)
    model_params_df.loc["hetrg_unobs", "theta_f2"] = optim_paras.item(17)
    model_params_df.loc["shares", "share_1"] = optim_paras.item(18)
    model_params_df.loc["shares", "share_2"] = optim_paras.item(19)
    model_params_df.loc["shares", "sigma_1"] = optim_paras.item(20)
    model_params_df.loc["shares", "sigma_2"] = optim_paras.item(21)
    model_params_df.loc["shares", "sigma_3"] = optim_paras.item(22)

    return model_params_df

Objective function

In [12]:
def get_objective(model_params_init_file_name, model_spec_init_file_name, moments_obs, weighting_matrix, optim_paras):

    # Update parameter
    model_params_df_updated = update_optim_paras(model_params_init_file_name, optim_paras)

    # Generate simulated data set
    data_frame_sim = soepy.simulate(model_params_df_updated, "model_spec_init.yml")
    
    # Process data frame
    data_frame_sim = pre_process_data(data_frame_sim)

    # Calculate simulated moments
    moments_sim = get_moments(data_frame_sim)

    # Obtain criterion function value
    func = calculate_criterion_func_value(moments_sim, moments_obs, weighting_matrix)
    
    print(func)

    return func

def calculate_criterion_func_value(moments_sim, moments_obs, weighting_matrix):

    # Move all moments from a dictionary to an array
    stats_obs, stats_sim = [], []

    for group in [
        "Wage_Distribution",
        "Choice_Probability",
    ]:
        for key_ in moments_obs[group].keys():
            stats_obs.extend(moments_obs[group][key_])
            stats_sim.extend(moments_sim[group][key_])

    # Construct criterion value
    stats_dif = np.array(stats_obs) - np.array(stats_sim)

    func = float(np.dot(np.dot(stats_dif, weighting_matrix), stats_dif))

    return func

In [19]:
import pybobyqa
from functools import partial

model_params_init_file_name = "toy_model_init_file_03_3types.pkl"
model_spec_init_file_name = "model_spec_init.yml"

data = pd.read_pickle("data_obs_3types.pkl")

data = pre_process_data(data)

moments_obs = get_moments(data)

weighting_matrix = get_weighting_matrix(data, num_agents_smm = 500, num_samples = 200)

objective = partial(get_objective, model_params_init_file_name, model_spec_init_file_name, moments_obs, weighting_matrix)

optim_paras_start = np.tile(
    (
        1.925,
        1.991,
        1.974,
        0.122,
        0.157,
        0.273,
        0.181,
        0.094,
        0.128,
        0.078,
        0.054,
        0.070,
        1.900,
        2.441,
        -0.200,
        -0.500,
        -0.100,
        -0.300,
        0.2,
        0.3,
        0.010,
        0.192,
        0.470,
    ),
    1,
)

lower = np.tile(
    (
        1.000,
        1.000,
        1.000,
        0.050,
        0.050,
        0.050,
        0.005,
        0.005,
        0.005,
        0.001,
        0.001,
        0.001,
        1.00,
        1.00,
        -0.400,
        -0.800,
        -0.400,
        -0.800,
        0.001,
        0.001,
        0.001,
        0.001,
        0.001,
    ),
    1,
)

upper = np.tile(
    (
        3.000,
        3.000,
        3.000,
        0.400,
        0.400,
        0.400,
        0.600,
        0.600,
        0.600,
        0.150,
        0.150,
        0.150,
        4.00,
        4.00,
        -0.050,
        -0.150,
        -0.050,
        -0.150,
        0.999,
        0.999,
        0.800,
        0.800,
        0.800,
    ),
    1,
)

# Log
logging.basicConfig(level=logging.INFO, format="%(message)s")

# Optimize
soln = pybobyqa.solve(
    objective,
    optim_paras_start,
    rhobeg=0.01,
    rhoend=1e-9,
    maxfun=5000,
    bounds=(lower, upper),
    scaling_within_bounds=True,
)

print(soln)

Function eval 1 at point 1 has f = 0 at x = [...]
Initialising (random directions)


0.0


Function eval 2 at point 2 has f = 0.544038210186347 at x = [...]


0.5440382101863469


Function eval 3 at point 3 has f = 0.2291975558896 at x = [...]


...

2.6655429135788105e-16


Function eval 297 at point 297 has f = 1.91648162331943e-16 at x = [...]


1.9164816233194344e-16


Function eval 298 at point 298 has f = 2.56024064251713e-16 at x = [...]
Did a total of 1 run(s)


2.560240642517127e-16
****** Py-BOBYQA Results ******
Solution xmin = [ 1.925  1.991  1.974  0.122  0.157  0.273  0.181  0.094  0.128  0.078
  0.054  0.07   1.9    2.441 -0.2   -0.5   -0.1   -0.3    0.2    0.3
  0.01   0.192  0.47 ]
Objective value f(xmin) = 0
Needed 298 objective evaluations (at 298 points)
Approximate gradient = [-2.70447649e-05  1.42401261e-06  7.67200202e-06  1.80599237e-04
  1.71684339e-04 -2.40230444e-04 -2.72614832e-05  1.46412532e-04
  1.12485076e-04  1.09471299e-03 -9.15517581e-05  5.97114421e-04
 -1.01721547e-05  1.62273270e-05  1.15671662e-04 -1.63874327e-04
 -2.54149754e-04 -2.69829418e-04  2.81805959e-05  1.28943922e-05
 -8.18993431e-05  8.03169250e-05 -4.60988670e-05]
Not showing approximate Hessian because it is too long; check self.hessian
Exit flag = 0
Success: rho has reached rhoend
******************************

